# The map of Per Capita Income per Riding for 2015 elections

Legend:

* Red - Liberal Party of Canada

* Blue - Conservative Party of Canada

* Orange - New Democratic Party

* Light Blue - Bloc Québécois

* Green  - Green Party of Canada

The relative size of the circle corresponds to the per capita income for the Riding.

**N.B.** How to calculate per capita income?
    For example, in final_elected.csv, for FED 10001, the population is 81540, but 68487 electors; in income.csv, 'Total' is 72030. I guess, the amount of Income was calculated for all the taxpayers (72030), that included those who didn't have the right to vote. 72030 - 68487 = 3543 That many???

In [4]:
# Dependencies and initialization
import gmaps
import pandas as pd

# Google developer API key
from api_keys import g_key

# Data sources
geodata_csv = 'geodata.csv' # FED Coordinates (key: 'FED Id')
parties_csv = '../data/winning_parties_2015.csv' # List of winning parties (key: 'Custom Name')
results_csv = '../data/final_elected.csv' # List of election results by FED (keys: 'FED ID', 'Party Won')
income_csv = '../data/income.csv' # FED Income Data (key: 'FED ID')

# Conclusions

**No conclusions yet.**

In [5]:
# Read data
geodata_df = pd.read_csv(geodata_csv).loc[:, ['FED Id', 'FED Name', 'Latitude', 'Longitude']]
parties_df = pd.read_csv(parties_csv).loc[:, ['Id', 'Custom Name']]
results_df = pd.read_csv(results_csv).loc[:, ['FED ID', 'Party Won']]
income_df = pd.read_csv(income_csv, encoding='windows-1252').loc[:, ['Prov/Terr', 'FED ID', 'Total', 'Net Income']]
income_df = income_df[income_df['Prov/Terr'] != 'TOTAL']
income_df['Per Capita'] = income_df['Net Income'] / income_df['Total']
income_df.head()

,Prov/Terr,FED ID,Total,Net Income,Per Capita
0,10,10001.0,72030,3.168392e+09,43987.116479
1,10,10002.0,64920,2.303218e+09,35477.788047
2,10,10003.0,65130,2.286474e+09,35106.310456
3,10,10004.0,20830,1.089412e+09,52300.144023
4,10,10005.0,77250,2.714495e+09,35139.093851


In [6]:
# Some useful functions

# From the pd.Series s with numeric values calculate scale for each point
def make_scale_list(s, bin_count=10):
    interval_index = pd.interval_range(start=s.min() * 0.999, end=s.max(), periods=bin_count)
    return [1 + 1 * int(interval_index.get_loc(val)) for val in s]

# From pd.Series s with Party Ids make a list of party colors
def make_color_list(s):
    # Source: https://www.rapidtables.com/web/color/RGB_Color.html
    c_blue      = (51, 51, 255)
    c_lightblue = (51, 153, 255)
    c_green     = (0, 204, 0)
    c_orange    = (255, 165, 0)
    c_red       = (255, 0, 0)

    # Source: https://en.wikipedia.org/wiki/Political_colour
    party_colors = {
        'CPC': c_blue,
        'BC' : c_lightblue,
        'GRN': c_green,
        'NDP': c_orange,
        'LIB': c_red
    }

    return s.map(lambda id: party_colors[id]).tolist()

In [10]:
# Create data set

df = pd.merge(geodata_df, results_df, how='left', left_on='FED Id', right_on='FED ID')
df = pd.merge(df, parties_df, how='left', left_on='Party Won', right_on='Custom Name')
df = pd.merge(df, income_df, how='left', left_on='FED Id', right_on='FED ID')
df.drop(columns=['FED ID_x', 'FED ID_y', 'Custom Name', 'Prov/Terr', 'Total', 'Net Income'], inplace=True)
df.rename(columns={'Id': 'Party Id'}, inplace=True)
#df = df[df['Party Id'] == 'NDP'] # Filter for testing
df.head()

,FED Id,FED Name,Latitude,Longitude,Party Won,Party Id,Per Capita
0,11001,Cardigan,46.251413,-62.652435,Liberal,LIB,38040.006673
1,11002,Charlottetown,46.267987,-63.143870,Liberal,LIB,37035.042159
2,24056,Pierrefonds--Dollard,45.491925,-73.855406,Liberal,LIB,41518.987342
3,24057,Pontiac,46.760139,-76.594503,Liberal,LIB,42695.359079
4,24058,Portneuf--Jacques-Cartier,47.137004,-71.836506,Conservative,CPC,43360.705780


In [12]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

# Customize the size of the figure
figure_layout = {
    'width': '900px',
    'height': '675px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
center=(43.6551165, -79.3869946) # Downtown Toronto

fig = gmaps.figure(layout=figure_layout, center=center, zoom_level=8)

# FED central points
locations = df[['Latitude', 'Longitude']]
hover_text = df['FED Name'].tolist()
fill_color = make_color_list(df['Party Id'])
stroke_color = fill_color
scale = make_scale_list(df.iloc[:, -1], bin_count=15)

data_layer = gmaps.symbol_layer(
    locations, # hover_text="hover_text", - doesn't work :(
    fill_color=fill_color, stroke_color=stroke_color,
    scale=scale
)

# Add the layer to the map
fig.add_layer(data_layer)

# Display
fig

Figure(layout=FigureLayout(border='1px solid black', height='675px', margin='0 auto 0 auto', padding='1px', wi…